# NIFTY50 PRICE PREDICTION

### This Program is made to predict the next days NIFTY50 index price using machine learning 

# Importing Useful libraries
### From sklearn - StandardScaler, mean_squared_error, r2_score, mean_absolute_error, train_test_split, LinearRegression, RandomForestRegressor, SVR, DecisionTreeRegressor and some tree libraries Lightgbm and Xgboost

### Pandas, Numpy, Joblib

In [45]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Loading Datasets
#### Converting Date column to datetime type
#### Setting index of DataFrame to Date column

In [46]:
data = pd.read_csv('../data/NIFTY50_data.csv', parse_dates=['Date'], index_col='Date')
data.sort_index(inplace=True)

# Feature Engineering

In [47]:
data['Return'] = data['Close'].pct_change()
data['Rel_Volume'] = data['Volume'] / data['Volume'].rolling(20).mean()
data['Return_10'] = data['Close'].pct_change(10)
data['Return_3'] = data['Close'].pct_change(3)
data['Volatility'] = data['Return'].rolling(20).std()
data['Volatility_5'] = data['Return'].rolling(5).std()
data['MA20'] = data['Close'].rolling(20).mean()
data['MA50'] = data['Close'].rolling(50).mean()
data['Trend20'] = (data['Close'] - data['MA20']) / data['MA20']
data['Trend50'] = (data['Close'] - data['MA50']) / data['MA50']
data['HL_Range'] = (data['High'] - data['Low']) / data['Close']
data['Gap'] = (data['Open'] - data['Close'].shift(1)) / data['Close'].shift(1)
data['Open'] = np.log(data['Open'])
data['High'] = np.log(data['High'])
data['Low'] = np.log(data['Low'])
data['Close'] = np.log(data['Close'])

# Calculate RSI
delta = data['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
# Wilder smoothing
avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

data['Target'] = data['Return'].shift(-1)  # Predict next day's return

data.drop(columns=['MA20', 'MA50'], inplace=True)
data.dropna(inplace=True)

# Splitting the Data into Train and Test

In [48]:
train_size = int(len(data) * 0.8)

y_train = data['Target'][:train_size]
y_test = data['Target'][train_size:]

data.drop(columns=['Target'], inplace=True)
X_train = data[:train_size]
X_test = data[train_size:]

# Creating Piplines to Train Different models and measure metrics

In [49]:
tree_models = {
    "rf": RandomForestRegressor(n_estimators=200, random_state=42),
    "xgb": XGBRegressor(n_estimators=300, max_depth=3, learning_rate=0.05, random_state=42),
    "lgbm": LGBMRegressor(n_estimators=300, max_depth=3, learning_rate=0.05, random_state=42)
}

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linear_models = {
    "lr": LinearRegression(),
    "svr": SVR(kernel='rbf')
}

### If I trade based on the prediction, how much money do I make? -> That’s strategy return.
### If prediction > 0 → go LONG
### If prediction < 0 → go SHORT

In [50]:
def print_metrics(name, y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    direction_acc = np.mean(np.sign(y_true) == np.sign(y_pred))
    
    position = np.sign(y_pred)              # long or short
    strategy_return = position * y_true     # actual profit
    cumulative_return = (1 + strategy_return).cumprod()

    all_metrics = {
        "MSE": mse,
        "MAE": mae,
        "Directional Accuracy": direction_acc,
        "Final Equity": cumulative_return.iloc[-1]
    }

    import json
    import os

    os.makedirs('../results', exist_ok=True)
    with open(f'../results/{name}_metrics.json', 'w') as f:
        json.dump(all_metrics, f, indent=4)


In [51]:
# Tree models
for name, model in tree_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_metrics(name, y_test, y_pred)

# Linear models
for name, model in linear_models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print_metrics(name, y_test, y_pred)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000758 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4080
[LightGBM] [Info] Number of data points in the train set: 2573, number of used features: 16
[LightGBM] [Info] Start training from score 0.000512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

# Fine Tuning LIGHTGBM model as it is best so far

In [52]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "n_estimators": [200, 300, 400],
    "learning_rate": [0.01, 0.02, 0.03],
    "max_depth": [-1, 3, 5],
    "num_leaves": [31, 50, 80],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "min_child_samples": [20, 30]
}

model = LGBMRegressor(random_state=42)

search = RandomizedSearchCV(
    model,
    param_grid,
    n_iter=30,
    scoring="neg_mean_squared_error",
    cv=3,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

best_model = search.best_estimator_

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000400 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4080
[LightGBM] [Info] Number of data points in the train set: 2573, number of used features: 16
[LightGBM] [Info] Start training from score 0.000512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [53]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print_metrics("best_lgbm", y_test, y_pred)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4080
[LightGBM] [Info] Number of data points in the train set: 2573, number of used features: 16
[LightGBM] [Info] Start training from score 0.000512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

# Pickling the model for production

In [54]:
import os

os.makedirs('../models', exist_ok=True)

joblib.dump(search.best_params_, '../models/best_lgbm_params.json')
joblib.dump(best_model, '../models/best_lgbm_model.pkl')

['../models/best_lgbm_model.pkl']

In [55]:
data.corr()

,Close,High,Low,Open,Volume,Return,Rel_Volume,Return_10,Return_3,Volatility,Volatility_5,Trend20,Trend50,HL_Range,Gap,RSI
Close,1.000000,0.999915,0.999912,0.999823,0.235586,0.012051,-0.056237,0.038631,0.020688,-0.195042,-0.164447,0.050058,0.079569,-0.150878,-0.010118,0.072395
High,0.999915,1.000000,0.999857,0.999922,0.239138,0.003207,-0.055065,0.033725,0.014415,-0.190831,-0.159572,0.043990,0.074452,-0.142489,-0.011541,0.067267
Low,0.999912,0.999857,1.000000,0.999900,0.231815,0.005309,-0.057147,0.039731,0.019023,-0.199503,-0.170104,0.051104,0.081573,-0.159200,-0.009188,0.072901
Open,0.999823,0.999922,0.999900,1.000000,0.236249,-0.003063,-0.055705,0.034797,0.012574,-0.195001,-0.164820,0.045054,0.076411,-0.149445,-0.009444,0.067751
Volume,0.235586,0.239138,0.231815,0.236249,1.000000,-0.020171,0.233111,-0.027094,-0.020150,0.354538,0.334781,-0.035839,-0.024439,0.393822,0.005461,-0.010220
Return,0.012051,0.003207,0.005309,-0.003063,-0.020171,1.000000,-0.036235,0.308420,0.572819,0.029088,0.024497,0.373267,0.230285,-0.138342,0.567002,0.320741
Rel_Volume,-0.056237,-0.055065,-0.057147,-0.055705,0.233111,-0.036235,1.000000,-0.028289,-0.027802,-0.036026,0.038158,-0.025136,-0.004800,0.130979,-0.021506,-0.013870
Return_10,0.038631,0.033725,0.039731,0.034797,-0.027094,0.308420,-0.028289,1.000000,0.547858,-0.068959,-0.328840,0.919438,0.676598,-0.359928,0.239861,0.762854
Return_3,0.020688,0.014415,0.019023,0.012574,-0.020150,0.572819,-0.027802,0.547858,1.000000,0.029170,-0.034211,0.619138,0.397671,-0.278345,0.366393,0.536261
Volatility,-0.195042,-0.190831,-0.199503,-0.195001,0.354538,0.029088,-0.036026,-0.068959,0.029170,1.000000,0.743612,-0.154748,-0.450125,0.539131,0.046829,-0.230884
